[Reference](https://configr.medium.com/building-an-expense-tracker-with-python-and-javascript-9276a9f7ba4c)

```
expense-tracker/
│
├── app.py
├── templates/
│   └── index.html
├── static/
│   ├── css/
│   │   └── styles.css
│   └── js/
│       └── scripts.js
├── db/
│   └── expenses.db
└── venv/
```

# Creating the Backend with Python

In [1]:
from flask import Flask, render_template, request, redirect, url_for
import sqlite3

app = Flask(__name__)

def init_db():
    with sqlite3.connect('db/expenses.db') as conn:
        cursor = conn.cursor()
        cursor.execute('''CREATE TABLE IF NOT EXISTS expenses (
                            id INTEGER PRIMARY KEY,
                            date TEXT,
                            category TEXT,
                            amount REAL,
                            description TEXT
                          )''')
        conn.commit()

@app.route('/')
def index():
    return render_template('index.html')

if __name__ == '__main__':
    init_db()
    app.run(debug=True)

# Handling User Inputs

In [4]:
from flask import Flask, render_template, request, redirect, url_for
import sqlite3

app = Flask(__name__)

def init_db():
    with sqlite3.connect('db/expenses.db') as conn:
        cursor = conn.cursor()
        cursor.execute('''CREATE TABLE IF NOT EXISTS expenses (
                            id INTEGER PRIMARY KEY,
                            date TEXT,
                            category TEXT,
                            amount REAL,
                            description TEXT
                          )''')
        conn.commit()

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/add', methods=['POST'])
def add_expense():
    if request.method == 'POST':
        date = request.form['date']
        category = request.form['category']
        amount = request.form['amount']
        description = request.form['description']

        with sqlite3.connect('db/expenses.db') as conn:
            cursor = conn.cursor()
            cursor.execute("INSERT INTO expenses (date, category, amount, description) VALUES (?, ?, ?, ?)",
                           (date, category, amount, description))
            conn.commit()

        return redirect(url_for('index'))

@app.route('/expenses')
def view_expenses():
    with sqlite3.connect('db/expenses.db') as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM expenses")
        expenses = cursor.fetchall()
    return render_template('expenses.html', expenses=expenses)

if __name__ == '__main__':
    init_db()
    app.run(debug=True)

# Designing the Frontend with HTML, CSS, and JavaScript
## Building the HTML Structure

```
# index.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Expense Tracker</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='css/styles.css') }}">
</head>
<body>
    <h1>Expense Tracker</h1>
    <form action="/add" method="POST">
        <label for="date">Date:</label>
        <input type="date" id="date" name="date" required>
        
        <label for="category">Category:</label>
        <input type="text" id="category" name="category" required>
        
        <label for="amount">Amount:</label>
        <input type="number" step="0.01" id="amount" name="amount" required>
        
        <label for="description">Description:</label>
        <input type="text" id="description" name="description">
        
        <button type="submit">Add Expense</button>
    </form>
    
    <a href="/expenses">View Expenses</a>
</body>
</html>
```

## Styling with CSS
```
body {
    font-family: Arial, sans-serif;
    background-color: #f4f4f4;
    color: #333;
    padding: 20px;
}

h1 {
    text-align: center;
}

form {
    max-width: 400px;
    margin: 0 auto;
    padding: 20px;
    background-color: #fff;
    border-radius: 8px;
    box-shadow: 0 2px 10px rgba(0, 0, 0, 0.1);
}

label {
    display: block;
    margin-bottom: 8px;
}

input {
    width: 100%;
    padding: 10px;
    margin-bottom: 15px;
    border: 1px solid #ddd;
    border-radius: 4px;
}

button {
    width: 100%;
    padding: 10px;
    background-color: #28a745;
    color: #fff;
    border: none;
    border-radius: 4px;
    cursor: pointer;
}

button:hover {
    background-color: #218838;
}

a {
    display: block;
    text-align: center;
    margin-top: 20px;
    color: #007bff;
}
```

## Creating Dynamic Visualizations with JavaScript
```
# static/js > scripts.js
<!-- Add this in your `expenses.html` file -->
<canvas id="expenseChart"></canvas>
<script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
<script src="{{ url_for('static', filename='js/scripts.js') }}"></script>

document.addEventListener("DOMContentLoaded", function() {
    // Fetch the expense data from the backend
    const expenseData = JSON.parse('{{ expenses | tojson | safe }}');
    
    const categories = {};

    // Categorize the expenses and sum the amounts per category
    expenseData.forEach(expense => {
        const category = expense[2]; // Assuming category is the third element in the expense array
        const amount = parseFloat(expense[3]); // Assuming amount is the fourth element in the expense array
        if (!isNaN(amount)) {
            categories[category] = (categories[category] || 0) + amount;
        }
    });
    
    // Prepare the data for the chart
    const chartData = {
        labels: Object.keys(categories),
        datasets: [{
            label: 'Spending by Category',
            data: Object.values(categories),
            backgroundColor: 'rgba(75, 192, 192, 0.2)',
            borderColor: 'rgba(75, 192, 192, 1)',
            borderWidth: 1
        }]
    };

    // Get the context of the canvas element and create the chart
    const ctx = document.getElementById('expenseChart').getContext('2d');
    new Chart(ctx, {
        type: 'bar',
        data: chartData,
        options: {
            scales: {
                y: {
                    beginAtZero: true
                }
            }
        }
    });
});
```

# Enhancing User Experience with Additional Features
## Enhancing User Experience with Additional Features


In [5]:
from flask import Flask, render_template, redirect, url_for, request, flash
from flask_sqlalchemy import SQLAlchemy
from flask_login import LoginManager, UserMixin, login_user, logout_user, login_required, current_user

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///db/expenses.db'
app.config['SECRET_KEY'] = 'your_secret_key'

db = SQLAlchemy(app)
login_manager = LoginManager(app)
login_manager.login_view = 'login'

class User(UserMixin, db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(150), unique=True, nullable=False)
    password = db.Column(db.String(150), nullable=False)

@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))

# Additional routes for user authentication (register, login, logout)

@app.route('/expenses', methods=['GET', 'POST'])
def view_expenses():
    query = "SELECT * FROM expenses WHERE user_id = ?"
    filters = [current_user.id]

    if request.method == 'POST':
        if request.form['start_date']:
            query += " AND date >= ?"
            filters.append(request.form['start_date'])
        if request.form['end_date']:
            query += " AND date <= ?"
            filters.append(request.form['end_date'])
        if request.form['category']:
            query += " AND category = ?"
            filters.append(request.form['category'])

    with sqlite3.connect('db/expenses.db') as conn:
        cursor = conn.cursor()
        cursor.execute(query, filters)
        expenses = cursor.fetchall()

@app.route('/export_csv')
@login_required
def export_csv():
    with sqlite3.connect('db/expenses.db') as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM expenses WHERE user_id = ?", (current_user.id,))
        expenses = cursor.fetchall()

    output = io.StringIO()
    writer = csv.writer(output)
    writer.writerow(['Date', 'Category', 'Amount', 'Description'])

    for expense in expenses:
        writer.writerow(expense[1:])

    output.seek(0)

    return Response(output, mimetype='text/csv',
                    headers={"Content-Disposition": "attachment;filename=expenses.csv"})

# Deploying Your Expense Tracker
```
heroku login
heroku create your-app-name
git push heroku main
heroku open
```